## Task: Convert pdf contents in to embeddings and store it in pinecone DB and Perform QA

In [4]:
# !pip install pinecone-client
# !pip install PyPDF
# !pip install tiktoken #For accessing openai embedding model class..

- do visit = https://app.pinecone.io/organizations/-O2do8KlggNowtR7GZb_/projects/9fbc0608-b462-486c-8535-26bbc9ec554a/indexes
- Signup using google(shreedharjagatap2002@gmai.com)
- Free tier is available with 2gb of storage , WUs = 2M , RUs= 1M
- go to api key -> they will provide you 1 by default key,,,,,,,,you can also create new api key....
- i will create new api key with name "test"..................
- there is a option "indexes"...in free tier you canc reate one index only........
- 

### 1. Import Lib's

In [4]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI  #Try gemini,lama,palm...etc
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

### 2. Load PDF

- create pdf_data foled,,and add shree_jagatap_analyst.pdf file in it,,bcz we are designing basic QA system

In [17]:
loader = PyPDFDirectoryLoader("pdf_data")

In [18]:
data = loader.load()

In [19]:
data
#len(data)-> 1,,,,bcz 1 page is there

[Document(metadata={'source': 'pdf_data\\shree_jagatap_analyst.pdf', 'page': 0}, page_content='Shreedhar Sanjay Jagatap  \nshreedharjagatap2002 @gmail.com | +91 9663659354  \nhttps://github.com/shreedhar13  | https://www.linkedin.com/in/shreedhar13/  \nLanguages: C, Python, SQL  \nTechnologies & Tools: Advance  Excel, PowerBI , MySQL,  Machine Learning , Deep Learning , NLP, LLM’s , LangChain, \nStatistic al Analysis , Mathamatics , Flask . \n \nWork Experience  \nAltizon , Pune  Jun 2023 – Sep 2023  \nData Science  (INTERN)  \n• Conducted extensive data preprocessing, including data cleaning, normalization, and feature engineering.  \n• Performed rigorous  data analysis and visualization using Pandas, NumPy, Matplotlib  and Seaborn.  Utilized Python \nand key ML libraries such as TensorFlow, Keras, and Scikit -learn  for various  regression and classification , ANN, CNN  \nand RNN  model building . \n• Performed hyperparameter tuning, cross -validation, and model optimization to enhan

In [20]:
data[0].page_content

'Shreedhar Sanjay Jagatap  \nshreedharjagatap2002 @gmail.com | +91 9663659354  \nhttps://github.com/shreedhar13  | https://www.linkedin.com/in/shreedhar13/  \nLanguages: C, Python, SQL  \nTechnologies & Tools: Advance  Excel, PowerBI , MySQL,  Machine Learning , Deep Learning , NLP, LLM’s , LangChain, \nStatistic al Analysis , Mathamatics , Flask . \n \nWork Experience  \nAltizon , Pune  Jun 2023 – Sep 2023  \nData Science  (INTERN)  \n• Conducted extensive data preprocessing, including data cleaning, normalization, and feature engineering.  \n• Performed rigorous  data analysis and visualization using Pandas, NumPy, Matplotlib  and Seaborn.  Utilized Python \nand key ML libraries such as TensorFlow, Keras, and Scikit -learn  for various  regression and classification , ANN, CNN  \nand RNN  model building . \n• Performed hyperparameter tuning, cross -validation, and model optimization to enhance performance . \n• Collaborated with cross -functional teams to align models with business n

### 3. Toenization

- This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order 
until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. **This has the effect of trying to keep all paragraphs (
and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically
related pieces of text**.

- i am not doing word wise tokenization,,,,,,,,i want sentence ie; want chucnk of data,,,,,

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=509 , chunk_overlap=20)

#chunk_size : refers to the maximum or atmost 500 characters in each chunk of text after splitting

#chunk_overlap: This specifies the number of characters that will overlap between consecutive chunks. 
#For example, if chunk_overlap is set to 20, the last 20 characters of one chunk will be repeated at the beginning of the next chunk.



In [22]:
text_chunk = text_splitter.split_documents(data)

In [23]:
text_chunk

[Document(metadata={'source': 'pdf_data\\shree_jagatap_analyst.pdf', 'page': 0}, page_content='Shreedhar Sanjay Jagatap  \nshreedharjagatap2002 @gmail.com | +91 9663659354  \nhttps://github.com/shreedhar13  | https://www.linkedin.com/in/shreedhar13/  \nLanguages: C, Python, SQL  \nTechnologies & Tools: Advance  Excel, PowerBI , MySQL,  Machine Learning , Deep Learning , NLP, LLM’s , LangChain, \nStatistic al Analysis , Mathamatics , Flask . \n \nWork Experience  \nAltizon , Pune  Jun 2023 – Sep 2023  \nData Science  (INTERN)'),
 Document(metadata={'source': 'pdf_data\\shree_jagatap_analyst.pdf', 'page': 0}, page_content='• Conducted extensive data preprocessing, including data cleaning, normalization, and feature engineering.  \n• Performed rigorous  data analysis and visualization using Pandas, NumPy, Matplotlib  and Seaborn.  Utilized Python \nand key ML libraries such as TensorFlow, Keras, and Scikit -learn  for various  regression and classification , ANN, CNN  \nand RNN  model bui

In [24]:
len(text_chunk)
#9 chunks are formed or 9 sentenecs/paragraphs are formed

9

In [25]:
print(text_chunk[0].page_content)

Shreedhar Sanjay Jagatap  
shreedharjagatap2002 @gmail.com | +91 9663659354  
https://github.com/shreedhar13  | https://www.linkedin.com/in/shreedhar13/  
Languages: C, Python, SQL  
Technologies & Tools: Advance  Excel, PowerBI , MySQL,  Machine Learning , Deep Learning , NLP, LLM’s , LangChain, 
Statistic al Analysis , Mathamatics , Flask . 
 
Work Experience  
Altizon , Pune  Jun 2023 – Sep 2023  
Data Science  (INTERN)


In [26]:
len(text_chunk[0].page_content)
#bcz this method  RecursiveCharacterTextSplitter , has the effect of trying to keep all paragraphs ( and then sentences, and then words) 
#together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

426

### 4. Using OpenAI Embeddings

- **you can use any other embedding model also (ex- word2vec,...etc)**

In [12]:
os.environ["OPENAI_API_KEY"] = "sk-proj-Ps4tuZe9Aig4mXbd389FT3BlbkFJ818rdfQAr1LXkfk8IA2B"
#set this variable into system environment..ie;inside environmental variables

In [13]:
embedding = OpenAIEmbeddings()
#embedding is object of OpenAIEmbeddings()

C:\Users\shree\.conda\envs\playing_with_genai\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [52]:
embedding.embed_query("How are you")
#Automatically,,OPENAI_API_KEY is accessed,,and request is done,,,,,you have to pay for it,,so use word2vec,,which is free,,,using genom library
#embedding will internally do word tokenization , and convert each word to vector and finally averaging them to form single vector which represents above sentence
#may be openai uses some diff aggregation in embeddings to convertmultiple vectors of the sentence to single vector,,,,see below how we done manually in w2v

In [ ]:
len(embedding.embed_query("How are you"))
#VVVVIIIIIIIIMMMMMMPPPPPPPPPP
#what dimensional you get,,exact same dimensional vector you will get for other sentences also,,no matter how big the sentence is,,,,,,

- some openai embedding model will not give you exactly 1600 dimensional vector for sentences.......................based on configuration and system compatibility it will give you diff length or diff dimensional vector also
- if u used high_end openai embedding model then it will give you 1600 dimensional vectors for sentences
- **ie; embedding.embed_query("How are you")  this embedding model gaved me 1536 dimensional vector**
- note:for this sentence we got 1536 dimensional vector,,then it is always true that we get same dimensioanl vector for all other sentences also,......
- it is not the case that,,,each sentence has diff -diff dimensional vectors........all sentences has same dimensional vector only.
- **same with Word2vec and all other embedding techniques/model**
- these dimensions nothing but features,,,,,and these features same for all vectors,,,so it has to be the same dimensional vectors.......read notes of w2v and openai embeddings you come to know

- this sentence is passes to openai embedding model through our OPENAI_API_KEY...................
- first this sentence is breakdown in to sub-words ,openai uses subword technique,,,see w2v vs openai embedding text file,,,,
- then each sub word is converted into vecto with 1600 dimension,,,if u used that model which uses 1600 dimension vector to represent each sentence/text
- then those subwords vector is aggregated(sum or average , ...etc),,to form 1 vector with 1600 dimensions
- and it will return this single vector with 1600 dimension ,, which represents this sentence "How are you"
- now you can store it in your vector databases or feed to any deep learning model like ANN,RNN.LSTM..etc......dont feed it to LLM model,,bcz it cant understand this,,,,,,,you can give text directly to LLM model,,it will internally do above steps to convert text into vectors and understand the meaning and give u answer

### 5. Pinecone connection and transfer embeddings,,and convert vector to equivavlent text we use RetrievalQA,,,which has openai embeddings and openai llm,which alter/modify the text according to question,,in usefull manner,,,,,,,,,,

In [12]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

- i have used "test: named api key,not using default one..go to pinecone -> api keys,,,u will see 2 keys 1) default 2)test,u can create manymore api keys,,but onlt=y 1 index in free tier

In [ ]:
#LEGACY approach
#pinecone.init(),,,is depricated,,,,,,,you have to use,,pc = Pinecone(api_key="76194580-c413-430a-892b-cd036d7844b9").see below in extras
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY' , '76194580-c413-430a-892b-cd036d7844b9')
PINCONE_API_ENV = OS.environ.get('PINECONE_API_KEY','')
pinecone.init(api_key = PINECONE_API_KEY , PINCONE_API_ENV )

index = pinecone.Index("shree_jagatap_analyst_resume_embeddings")#accessing index from pinecone
docsearch = Pinecone.from_texts(t.page_content for t in text_chunk) , embedding , index_name=index_name)#docsearch is Pinecone onject 
#which pointing to this "shree_jagatap_analyst_resume_embeddings" index present in my Pinecone DB account  

#Open Pinecone Db website,,and u can see your (shree_jagatap_analyst.pdf)document is added in the form of embeddings.....and on top index name is there........"shree_jagatap_analyst_resume_embeddings"
#now u can able to perform ,, similarity search...etc vector operations(add,sub,multiply,divide..etc)

In [20]:
query = "what is education of shreedhar?"

In [ ]:
docs=docsearch.similarity_search(query)
docs
#gives you most similar vector,,,iemost similar vectors with queried vector ,
#but to get text from this vector we use RetrievalQA mehtod

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm , chain_type="stuff" , retriever=docsearch.as_retriever())
#this code will convert vector to text,,,,,which is difficult in word2vec

In [ ]:
qa.run(query)
#Most similar text is displayed...(internall that most similar vector is coverted to text)

In [ ]:
#Note-> openai embeddings is reversible also,ie;after searching similar vector in pinecne,,you get that vector,and to convert it to text we use RetrievalQA method,,
#which gives output in text

# *******************************************************************************

# EXTRAS/experiments

### another method,here i am using Word2vec and pinecone only,,,not using any LLM

In [5]:
import numpy as np
from gensim.models import Word2Vec,KeyedVectors
import pinecone
from pinecone import Pinecone, ServerlessSpec

In [2]:
model =  KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz" , binary=True)

In [7]:
pc = Pinecone(api_key="76194580-c413-430a-892b-cd036d7844b9")

In [8]:
pc.list_indexes()
#as u see,,we have not created any indexes

{'indexes': []}

In [ ]:
# Go to Pinecone website -> create index with name "shree-jagatap-analyst-resume-embeddings" -> selected aws and us-east-1 region.
#now u can access it,,and upsert vectors
#in free tier u can create only one index,so below u see ,, only one dict is returned

In [10]:
pc.list_indexes()
#'dimension': 300, -> each vector dimension,,,,,ie; all vectors is of 300 dimensional

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 300,
              'host': 'shree-jagatap-analyst-resume-embeddings-k0vovtd.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'shree-jagatap-analyst-resume-embeddings',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [13]:
#select or access index in wich you want to upsert data
index_name = pc.list_indexes()[0]['name']

'shree-jagatap-analyst-resume-embeddings'

In [16]:
#index is a pinecone object which connects with "shree-jagatap-analyst-resume-embeddings",,remotely
index = pc.Index('shree-jagatap-analyst-resume-embeddings')

In [27]:
text_chunk

[Document(metadata={'source': 'pdf_data\\shree_jagatap_analyst.pdf', 'page': 0}, page_content='Shreedhar Sanjay Jagatap  \nshreedharjagatap2002 @gmail.com | +91 9663659354  \nhttps://github.com/shreedhar13  | https://www.linkedin.com/in/shreedhar13/  \nLanguages: C, Python, SQL  \nTechnologies & Tools: Advance  Excel, PowerBI , MySQL,  Machine Learning , Deep Learning , NLP, LLM’s , LangChain, \nStatistic al Analysis , Mathamatics , Flask . \n \nWork Experience  \nAltizon , Pune  Jun 2023 – Sep 2023  \nData Science  (INTERN)'),
 Document(metadata={'source': 'pdf_data\\shree_jagatap_analyst.pdf', 'page': 0}, page_content='• Conducted extensive data preprocessing, including data cleaning, normalization, and feature engineering.  \n• Performed rigorous  data analysis and visualization using Pandas, NumPy, Matplotlib  and Seaborn.  Utilized Python \nand key ML libraries such as TensorFlow, Keras, and Scikit -learn  for various  regression and classification , ANN, CNN  \nand RNN  model bui

In [43]:
sentence_vectors=list()
for s in text_chunk:
    words = str(s).lower().split() #bcz s,,is a langchain.core type,,,thus u cant able to perfom lower,,so convert it to text
    word_vectors = [model[word] for word in words if word in model] 
    sentence_vectors.append(np.mean(word_vectors, axis=0) )


In [45]:
len(sentence_vectors)
#we got 9 vectors for our 9 sentences

9

In [86]:
sentences=list()
for i in range(len(text_chunk)):
    sentences.append(str(text_chunk[i].page_content).lower())  
sentences

['shreedhar sanjay jagatap  \nshreedharjagatap2002 @gmail.com | +91 9663659354  \nhttps://github.com/shreedhar13  | https://www.linkedin.com/in/shreedhar13/  \nlanguages: c, python, sql  \ntechnologies & tools: advance  excel, powerbi , mysql,  machine learning , deep learning , nlp, llm’s , langchain, \nstatistic al analysis , mathamatics , flask . \n \nwork experience  \naltizon , pune  jun 2023 – sep 2023  \ndata science  (intern)',
 '• conducted extensive data preprocessing, including data cleaning, normalization, and feature engineering.  \n• performed rigorous  data analysis and visualization using pandas, numpy, matplotlib  and seaborn.  utilized python \nand key ml libraries such as tensorflow, keras, and scikit -learn  for various  regression and classification , ann, cnn  \nand rnn  model building . \n• performed hyperparameter tuning, cross -validation, and model optimization to enhance performance .',
 '• collaborated with cross -functional teams to align models with busine

In [87]:
len(sentences)

9

In [88]:
vectors_to_insert_with_text = [
    {"id": str(i), "values": vec.tolist(), "metadata": {"sentence": sent}} 
    for i, (vec, sent) in enumerate(zip(sentence_vectors, sentences), start=1)
]

In [89]:
vectors_to_insert_with_text

[{'id': '1',
  'values': [-0.05821533128619194,
   0.04375610500574112,
   0.01937704160809517,
   0.06714000552892685,
   -0.05788879469037056,
   0.0375518798828125,
   -0.06837157905101776,
   -0.08786620944738388,
   0.03796844556927681,
   0.0062622069381177425,
   -0.02837524376809597,
   -0.08675537258386612,
   -0.04624328762292862,
   -0.020617198199033737,
   -0.118560791015625,
   0.04929199069738388,
   -0.03901367262005806,
   0.11824341118335724,
   0.01972637139260769,
   -0.15627440810203552,
   -0.07811584323644638,
   -0.01841430738568306,
   0.0032707215286791325,
   0.04163818433880806,
   0.01326599158346653,
   -0.0015670775901526213,
   -0.08422622829675674,
   0.05213012546300888,
   -0.06782855838537216,
   -0.10185547173023224,
   0.0009765625,
   -0.03211364895105362,
   0.0007217407110147178,
   -0.054656982421875,
   -0.06080322340130806,
   0.0028228759765625,
   -0.06628493964672089,
   0.04558715969324112,
   0.09057922661304474,
   -0.03484616428613663,

In [90]:
index.upsert(vectors=vectors_to_insert_with_text)

{'upserted_count': 9}

# Querying/Question

In [100]:
#Example 1
query_vector = vectors_to_insert_with_text[0]['values']
query_vector
#query this vector,,,and get most similar in result

[-0.05821533128619194,
 0.04375610500574112,
 0.01937704160809517,
 0.06714000552892685,
 -0.05788879469037056,
 0.0375518798828125,
 -0.06837157905101776,
 -0.08786620944738388,
 0.03796844556927681,
 0.0062622069381177425,
 -0.02837524376809597,
 -0.08675537258386612,
 -0.04624328762292862,
 -0.020617198199033737,
 -0.118560791015625,
 0.04929199069738388,
 -0.03901367262005806,
 0.11824341118335724,
 0.01972637139260769,
 -0.15627440810203552,
 -0.07811584323644638,
 -0.01841430738568306,
 0.0032707215286791325,
 0.04163818433880806,
 0.01326599158346653,
 -0.0015670775901526213,
 -0.08422622829675674,
 0.05213012546300888,
 -0.06782855838537216,
 -0.10185547173023224,
 0.0009765625,
 -0.03211364895105362,
 0.0007217407110147178,
 -0.054656982421875,
 -0.06080322340130806,
 0.0028228759765625,
 -0.06628493964672089,
 0.04558715969324112,
 0.09057922661304474,
 -0.03484616428613663,
 0.03110961988568306,
 0.0540924072265625,
 0.03349762037396431,
 0.03435821458697319,
 0.000798034656

In [102]:
index.query(
    
    vector=query_vector,
    top_k=2,#top 2 matched/most similar vectors
    include_values=True,
    include_metadata=True,
    
)

{'matches': [{'id': '1',
              'metadata': {'sentence': 'shreedhar sanjay jagatap  \n'
                                       'shreedharjagatap2002 @gmail.com | +91 '
                                       '9663659354  \n'
                                       'https://github.com/shreedhar13  | '
                                       'https://www.linkedin.com/in/shreedhar13/  \n'
                                       'languages: c, python, sql  \n'
                                       'technologies & tools: advance  excel, '
                                       'powerbi , mysql,  machine learning , '
                                       'deep learning , nlp, llm’s , '
                                       'langchain, \n'
                                       'statistic al analysis , mathamatics , '
                                       'flask . \n'
                                       ' \n'
                                       'work experience  \n'
             

In [106]:
#Example 2
sentence = "which data analytics project he has done?"
words = sentence.lower().split()

# Step 2: Get word vectors and average them
word_vectors = [model[word] for word in words if word in model] 
sentence_vector = np.mean(word_vectors, axis=0)
query2_vector = sentence_vector.tolist()
query2_vector
#I have done bank loan analysis project,,and i have internship in data analytics,,,,,,,,,,,so these are expected result

[0.025146484375,
 -0.0451863594353199,
 0.0332082100212574,
 -0.02034505270421505,
 0.0068613686598837376,
 -0.0364176444709301,
 0.0005391438608057797,
 -0.0611775703728199,
 0.1068623885512352,
 -0.010823567397892475,
 -0.070770263671875,
 -0.0916951522231102,
 -0.01824331283569336,
 -0.046630859375,
 -0.0908203125,
 -0.0028889973182231188,
 0.0395100899040699,
 0.015528361313045025,
 -0.0971171036362648,
 -0.1368408203125,
 -0.1029256209731102,
 0.0956064835190773,
 -0.0314534492790699,
 0.0513712577521801,
 0.0010579427471384406,
 -0.0601603202521801,
 -0.0547281913459301,
 0.0670166015625,
 -0.0453694649040699,
 -0.085296630859375,
 0.043426513671875,
 0.0517171211540699,
 -0.1003621444106102,
 -0.002227783203125,
 0.02913411520421505,
 -0.0469970703125,
 -0.030975341796875,
 -0.0638020858168602,
 0.10788217931985855,
 -0.02046712301671505,
 0.0437418632209301,
 0.0774739608168602,
 0.0910542830824852,
 0.06709098815917969,
 -0.02414957620203495,
 -0.0812733992934227,
 -0.12589518

In [107]:
index.query(
    
    vector=query2_vector,
    top_k=2,
    include_values=True,
    include_metadata=True,
    
)
#as u see,,,bank loan analysis,,is most similar,,which is expected one and we get that only...........

{'matches': [{'id': '9',
              'metadata': {'sentence': 'detailed insights into loan data, '
                                       'borrower profiles, and loan '
                                       'performance metrics.dashboard '
                                       'integrates various \n'
                                       'visualizations and interactive '
                                       'elements to facilitate comprehensive '
                                       'data analysis. supporting informed '
                                       'decision -making \n'
                                       'and improving the overall loan '
                                       'management process by 50%.  \n'
                                       ' \n'
                                       'awards and certificates  \n'
                                       '• google data analytics specialization '
                                       '(coursera)  \n'
        

## see,,,,,,,,what text that similar vector represting ,we get that only,,,,,,,,,,,,,,,,,but if i want specif info w.r.t what question we asked then use LLM,,,which can extract particular info from similar sentence w.r.t asked question.

- ie; i asked what project i have done in data analytics?,,,,,,,answer should be Bank loan analysis , fassos data analysis , internship........but i am getting complete senteces/text which contains this,,,so i have to manually read it........................so to extract just Bank loan analysis , fassos data analysis , internship   use LLM
- **VVVVVVVIIIIIIIIMMMMMMP**

# *********************************************************************************************************************************************************

## Demonstartion of word2vec model

### Using word2vec and pinecone only,not using any llm

In [14]:
import numpy as np
from gensim.models import Word2Vec,KeyedVectors

In [15]:
## Load the Word2Vec model from the saved file
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz" , binary=True)
#now this model contains 3 million words , vector/embedding representation.....each word is represented with 300 dimensional vector or embeddings.......
#we have downloaded w2v model which represent each word in 300 dimensional vector

#OpenAI also have many embeddings model,,,in that High-end embedding model use 1600 dim vector for representing each word,
#other embeddings models from openai can represent 1 word in less thatn 1600 dimension vector,so read documentation of each embedding model and use it

In [16]:
model['man']
#If this word present,,then only it will display vector of it,,,else keyerror,,,,so use this code
#if 'man' in model:
#    model['man']
#else:
#    print('vector representation for this word is not present')


array([ 0.32617188,  0.13085938,  0.03466797, -0.08300781,  0.08984375,
       -0.04125977, -0.19824219,  0.00689697,  0.14355469,  0.0019455 ,
        0.02880859, -0.25      , -0.08398438, -0.15136719, -0.10205078,
        0.04077148, -0.09765625,  0.05932617,  0.02978516, -0.10058594,
       -0.13085938,  0.001297  ,  0.02612305, -0.27148438,  0.06396484,
       -0.19140625, -0.078125  ,  0.25976562,  0.375     , -0.04541016,
        0.16210938,  0.13671875, -0.06396484, -0.02062988, -0.09667969,
        0.25390625,  0.24804688, -0.12695312,  0.07177734,  0.3203125 ,
        0.03149414, -0.03857422,  0.21191406, -0.00811768,  0.22265625,
       -0.13476562, -0.07617188,  0.01049805, -0.05175781,  0.03808594,
       -0.13378906,  0.125     ,  0.0559082 , -0.18261719,  0.08154297,
       -0.08447266, -0.07763672, -0.04345703,  0.08105469, -0.01092529,
        0.17480469,  0.30664062, -0.04321289, -0.01416016,  0.09082031,
       -0.00927734, -0.03442383, -0.11523438,  0.12451172, -0.02

In [17]:
len(model['man'])
#as u see each word is represented with 300 dimensional vector,,,,,,,

300

In [18]:
# Step 1: Load Word2Vec model and preprocess sentence
# model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz" , binary=True)
sentence = "How are you bro"
words = sentence.lower().split()

# Step 2: Get word vectors and average them
word_vectors = [model[word] for word in words if word in model] # [[embeddings for how],[embeddings for are],[embeddings for you],[embeddings for bro]]
sentence_vector = np.mean(word_vectors, axis=0) # averaging 4 embeddings to get,,single vector or embedding for above sentence,,and you can store it in pinecone
sentence_vector

array([ 6.84814453e-02,  2.78930664e-02,  1.15600586e-01,  1.32080078e-01,
       -1.27197266e-01,  1.28906250e-01, -1.96228027e-02, -6.99157715e-02,
        7.35473633e-02, -2.05307007e-02, -4.67529297e-02, -1.15539551e-01,
       -1.37390137e-01,  5.24902344e-03, -2.55981445e-01,  8.41064453e-02,
        2.65502930e-03,  1.36596680e-01, -3.54614258e-02, -6.34765625e-03,
       -3.28979492e-02,  4.36706543e-02,  1.50527954e-01, -2.44140625e-02,
        6.52465820e-02,  8.88977051e-02, -6.93969727e-02, -4.20379639e-03,
        4.76264954e-02, -6.39648438e-02, -1.34277344e-02,  9.83886719e-02,
       -9.21020508e-02, -5.47981262e-02, -5.08422852e-02,  9.27429199e-02,
        1.21520996e-01,  4.76684570e-02,  1.32324219e-01,  1.08062744e-01,
        1.47644043e-01, -9.53063965e-02,  1.40533447e-01,  8.35342407e-02,
        9.24072266e-02, -1.34887695e-02, -3.54690552e-02, -2.04162598e-02,
        5.31616211e-02, -6.88476562e-02, -7.84301758e-02,  1.29882812e-01,
        1.54418945e-02, -

In [47]:
sentence="how are you"
words = sentence.lower().split()
word_vectors = [model[word] for word in words if word in model] 
print(word_vectors[0])

[ 0.26953125  0.0859375   0.09423828  0.04101562 -0.18359375  0.12890625
  0.0625     -0.10058594  0.1484375  -0.06591797  0.13671875 -0.19042969
 -0.12255859 -0.05908203 -0.29492188  0.08496094 -0.02258301  0.20410156
 -0.05151367 -0.03295898 -0.08056641  0.02624512  0.02758789  0.10107422
 -0.01708984  0.10693359  0.07324219 -0.11425781  0.02111816  0.06176758
 -0.11035156  0.29296875 -0.08789062 -0.19140625  0.05834961  0.12695312
  0.22949219  0.02575684  0.03344727  0.0255127   0.02624512  0.0279541
  0.04711914 -0.0456543  -0.06835938  0.05981445 -0.00946045  0.00500488
 -0.0144043   0.08300781  0.0625      0.03955078  0.03515625 -0.17675781
  0.08300781 -0.06738281 -0.14550781  0.07373047  0.18847656  0.0559082
  0.09130859  0.06933594 -0.15136719 -0.02307129 -0.07275391 -0.18457031
  0.06201172  0.125      -0.01055908 -0.02038574  0.12597656  0.09960938
  0.16699219 -0.16210938 -0.00430298 -0.14160156  0.15722656  0.38867188
 -0.12988281  0.0703125  -0.01470947 -0.01190186 -0.0

In [19]:
len(sentence_vector)

300

## VVVIIIMMPP

#### In openai embedding model above step 2 will be done by openai embedding model itself,,,,,,,,,,,,,but you have to do manually in word2vec,and it contains 3miilion words vectors,,and it cant handle out of vocabulary words,,and gives keyerror,but openai can handle out of vocab words very well it bcz it uses BPE technique,,read googlesWord2vec_embedding_model_VS_Openais_embedding_model

In [ ]:

# Step 3: Initialize Pinecone
pinecone.init(api_key='your-pinecone-api-key')
index_name = 'shree_jagatap_analyst_resume_embeddings'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(sentence_vector),  # Dimensionality of Word2Vec vectors
        metric='cosine',
        spec=pinecone.ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
#accessing index or connecting with index
index = pinecone.Index(index_name)

# Step 4: Insert the sentence vector into Pinecone
index.upsert([(sentence, sentence_vector.tolist())])

print(f"Inserted vector for sentence: '{sentence}'")

### Entire process looks like 

- **NOTE-> It is not necessary to use only OpenAI's LLM if you have used OpenAI embeddings. You can use any LLM, such as Gemini,mistral,llama...etc along with OpenAI embeddings:**
- **delete "shree-jagatap-analyst-resume-embeddings" from pinecone,,and create other index,,bcz in free tier you can create only one index at a time..........play with other documents,,,,and use any opensource llm (palm,llama..etc) to get exact answer,,what we are not getting from just w2v and pinecone**